In [2]:
import requests, re
from bs4 import BeautifulSoup
import glob

In [3]:
def retrieve_dinamalar_article(article_id):
    base_url='http://www.dinamalarnellai.com/web/news'
    url='{}/{}'.format(base_url,article_id)
    r=requests.get(url)
    html = r.text

    selector = 'body > div.container > div.dinamalar_middle_container_news > div'    
    pub_selector = 'body > div.container > div.dinamalar_middle_container_news > div > div > div > h4'
    text_selector = 'div.dinamalar_middle_container_news > div > div > p'
    #img_selector = 'div.dinamalar_middle_container_news > div > div > p > img'
    img_pattern='<img class="(.*?)" src="http://www.dinamalarnellai.com/site/news_folder/(.*?).jpg" style="(.*?)">'

    soup = BeautifulSoup(html, 'lxml')
    try:
        title_selector = 'body > div.container > div.dinamalar_middle_container_news > div > h1'
        title = soup.select(title_selector)[0].text
    except:
        title_pattern = '<h1(.*?)>\n(.*?)</h1>'
        title_class,title=re.findall(title_pattern, html)[0]
        title=title.replace('\t','')
        #print(title)
        assert title!=''
    try:
        pubdate = soup.select(pub_selector)[0].text.strip()
    except:
        pubdate = re.findall('<p class="pageMeta">(.*?),', html)[0]
        pubdate=pubdate.replace('\t','')
    text = '\n\n'.join([para.text.replace('\xa0',' ') for para in soup.select(text_selector)])
    if text=='':
        paras =soup.select('div > p') #, attrs={'class':'pageContent'})
        text = '\n\n'.join([para.text.replace('\xa0',' ') for para in paras])
   
    #imglinks = soup.select(img_selector)
    imglinks=re.findall(img_pattern, html)
    
    clean_title=title.replace(' ','_').replace('__','_').replace('..','').replace("'",'')
    clean_title=clean_title.replace('"','')
    for c in '‘’*,();!:?./':
        clean_title=clean_title.replace(c,'')
    clean_title='_'.join(clean_title.split('_')[:15])
    fname='dinamalar_nellai_'+str(article_id)+'_'+clean_title+'.txt'
    with open('articles/'+fname, 'w', encoding='utf-8') as f:
        f.write(title+'\n\n')
        f.write(url+'\n\n')
        f.write(pubdate+'\n\n')
        for img in imglinks:
            a,b,c=img    
            imgurl='http://www.dinamalarnellai.com/site/news_folder/'+b+'.jpg'
            f.write(imgurl+'\n\n')
        f.write(text+'\n\n')
    print()
    #print(title)
    #print(text[:200])
    print('saving',fname)
    #for img in soup.select(img_selector):
    for img in imglinks:
        a,b,c=img
        #print(b)
        #imgfilename=re.findall('http://www.dinamalarnellai.com/site/news_folder/(.*?).jpg',img['src'])[0]
        imgfilename=b
        imgurl='http://www.dinamalarnellai.com/site/news_folder/'+b+'.jpg'
        #imgfilename=title.replace(' ','_').replace('..','').replace('!','').replace(':','')+imgfilename.replace(' ','_').replace('(','').replace(')','')+'.jpg'
        article_id_s=str(article_id)
        imgfilename_clean=imgfilename.replace(' ','_').replace('(','').replace(')','').replace('.','').replace(',','')
        imgfilename=article_id_s+'_'+clean_title[:20]+'_'+imgfilename_clean+'.jpg'
        #print('saving', imgfilename)
        try:
            with open('images/'+imgfilename,'wb') as i:
                r2=requests.get(imgurl)
                i.write(r2.content)
        except:
            print(imgfilename,'write failed!')

In [4]:
import os
if not os.path.exists('articles'):
    os.makedirs('articles')
if not os.path.exists('images'):
    os.makedirs('images')

In [5]:
#86431 tms bhagavatar
#87752 kannadasan
#retrieve_dinamalar_article(87752) 

In [ ]:
textfiles=glob.glob('articles/*')
finished=[int(f.split("\\")[1].split('_')[2]) for f in textfiles]
good_retrievals=sorted(finished,reverse=True)
print(len(good_retrievals), 'files already found!')
for article_id in range(70000,60000,-1):
    if article_id in good_retrievals:
        #print(article_id,'already retrieved!')
        continue
    #print()
    #print(article_id)
    try:
        retrieve_dinamalar_article(article_id) 
    except:
        with open('misses.txt','a') as f:
            f.write(str(article_id)+' failed!/n')

24522 files already found!

saving dinamalar_nellai_64084_தற்போதைய_செய்தி_2.txt

saving dinamalar_nellai_63178_ஸ்டெர்லைட்_விவகாரம்_பசுமை_தீர்ப்பாயத்தின்_உத்தரவை_எதிர்த்து_தமிழக_அரசு_உச்சநீதிமன்றத்தில்_மேல்_முறையீடு.txt

saving dinamalar_nellai_63177_02012019_பருப்பு_மற்றும்_மாவு_எண்ணெய்_வகைகள்_மொத்த_மார்க்கெட்_விலை.txt

saving dinamalar_nellai_63176_சட்டசபையில்_இருந்து_திமுக_-_காங்கிரஸ்_வெளிநடப்பு-_வெளிநடப்பு_குறித்து_ஸ்டாலின்_விளக்கம்.txt

saving dinamalar_nellai_63175_02012019_இந்திய_ரூபாய்_நாணய_மாற்று_விகிதம்_துவக்க_நிலவரம்.txt

saving dinamalar_nellai_63174_காவிரி_விவகாரம்-_மாநிலங்களவையில்_பதாகைகள்_ஏந்தி_அதிமுக_எம்பிக்கள்_போராட்டம்.txt

saving dinamalar_nellai_63173_தமிழ்நாடு_சட்டசபை_கூடியது_துவக்கி_வைத்து_ஆளுநர்_பன்வாரிலால்_உரையாற்றினார்.txt

saving dinamalar_nellai_63172_அமெரிக்க_அரசு_நிர்வாகத்தின்_முடக்கத்தை_முடிவுக்கு_கொண்டு_வர_ஜனநாயக_கட்சியினர்_விருப்பம்.txt

saving dinamalar_nellai_63171_சபரிமலையில்_பெண்களை_அனுமதிக்க_வலியுறுத்தி_கேரளாவில்_பெண்கள்_மனித_சுவர்_போராட்டம்.txt

sav


saving dinamalar_nellai_63095_சீக்கிய_வன்முறை_வழக்கில்_ஆயுள்_தண்டனை_விதிக்கப்பட்ட_சஜ்ஜன்_குமார்_நீதிமன்றத்தில்_சரண்.txt

saving dinamalar_nellai_63094_காட்டுத்தனமான_பொருள்.txt

saving dinamalar_nellai_63093_விசேஷித்த_ஆவி.txt

saving dinamalar_nellai_63092_இந்திய_குடியரசு_தலைவர்_மற்றும்_பிரதமருக்கு_ரஷ்ய_அதிபர்_புதின்_புத்தாண்டு_வாழ்த்து.txt

saving dinamalar_nellai_63091_தேடினால்_வெளிப்படுவார்.txt

saving dinamalar_nellai_63090_ஆண்டவரை_நம்பினால்_நடக்காததும்_நடக்கும்.txt

saving dinamalar_nellai_63089_அன்புக்கு_இவர்_அடிமை.txt

saving dinamalar_nellai_63088_பெருமானாரின்_அருமை_மனைவி_கதீஜா_நாயகி_வாழ்க்கை_வரலாறு–_7.txt

saving dinamalar_nellai_63087_நபிமார்கள்_வரலாறு_230_ஹஜ்ரத்_ஈஸா_நபி_அலைஹிஸ்ஸலாம்.txt

saving dinamalar_nellai_63086_அரசியலுக்காக_நாட்டு_மக்களை_ஏமாற்றுக்கிறார்_பிரதமர்_மோடி_கர்நாடக_முதல்வர்_குமாரசாமி_குற்றச்சாட்டு.txt

saving dinamalar_nellai_63085_கஜா_புயல்_நிவாரணத்துக்கு_மத்திய_உள்துறை_ரூ_114612_கோடி_ஒதுக்கீடு.txt

saving dinamalar_nellai_63084_நபிமார்களின்_உடலை_மண்_தீண்டாது


saving dinamalar_nellai_63010_வெளிநாட்டு_ஆன்லைன்_வர்த்தக_நிறுவனங்கள்_மீதான_கட்டுப்பாடுகள்_இந்தியாவின்_அந்நிய_நேரடி_முதலீடுகளை_பாதிக்கும்_என_எச்சரிக்கை.txt

saving dinamalar_nellai_63009_சீனாவில்_நிலக்கரி_சுரங்கத்தில்_மண்_சரிவில்_சிக்கி_5_பேர்_பலி.txt

saving dinamalar_nellai_63008_கொத்தவால்சாவடியில்_கடத்தப்பட்ட_சிறுமி_–_சென்னை_சென்ட்ரல்_ரயில்_நிலையத்தில்_மீட்பு.txt

saving dinamalar_nellai_63007_எகிப்தில்_உள்ள_பிரமிடுகள்_அருகே_குண்டுவெடிப்பு_வியட்நாம்_சுற்றுலாப்_பயணிகள்_3_பேர்_பலி.txt

saving dinamalar_nellai_63006_தமிழ்நாடு_புதுச்சேரியில்_24_மணி_நேரத்தில்_மழை_பெய்ய_வாய்ப்பு_-_வானிலை_ஆய்வு_மையம்.txt

saving dinamalar_nellai_63005_ஜெயலலிதாவுக்கு_அளிக்கப்பட்ட_சிகிச்சையில்_சதி_ராம்மோகன்ராவ்_ராதாகிருஷ்ணனை_எதிர்_மனுதாரர்களாக_சேர்க்கக்_கோரிக்கை.txt

saving dinamalar_nellai_63004_சாதி_மறுப்பு_திருமணங்களை_ஆதரிக்கவேண்டும்_ஜிக்னேஷ்_மேவானி_வேண்டுகோள்.txt

saving dinamalar_nellai_63003_உழவர்கள்_மீது_அடக்குமுறையை_திணிக்கக்_கூடாது_மத்திய_மாநில_அரசுகளுக்கு_ராமதாஸ்_கண்டனம்.txt

saving dinamalar_nella


saving dinamalar_nellai_62922_வங்­கா­ள­தேச_மக்­க­ளின்_நம்­பிக்­கையை_இந்­தியா_பெற_வேண்­டும்.txt

saving dinamalar_nellai_62921_வாகனங்களுக்கு_உயர்_பாதுகாப்பு_அம்சங்கள்_கொண்ட_நம்பர்_பிளேட்டுகள்_2019_ஏப்ரல்_முதல்_கட்டாயம்_நிதின்_கட்கரி_தகவல்.txt

saving dinamalar_nellai_62920_கடவுள்_ராமருக்கு_வீடு_பாஜக_எம்பி_கலெக்டரிடம்_மனு.txt

saving dinamalar_nellai_62919_புயல்_நிவாரணம்_மற்றும்_சீரமைப்பு_பணிகளின்_நிலை_அமைச்சர்களுடன்_முதல்வர்_ஆலோசனை.txt

saving dinamalar_nellai_62918_வெங்காயம்_உருளைக்கிழங்கு_விலை_வீழ்ச்சி_தேர்தலில்_மோடிக்கு_பாடம்_புகட்டுவோம்_விவசாயிகள்_சபதம்.txt

saving dinamalar_nellai_62917_பிடித்த_படம்.txt

saving dinamalar_nellai_62916_நண்பர்கள்.txt

saving dinamalar_nellai_62915_அஜீத்_பட_வில்லன்_மோதல்.txt

saving dinamalar_nellai_62914_உத்தரபிரதேசத்தில்_வன_விலங்குகளை_வேட்டையாடிய_5_பேர்_கைது.txt

saving dinamalar_nellai_62913_என்_காதலி_சீன்_போடுறா.txt

saving dinamalar_nellai_62912_காணாமல்_போன_பெட்டிக்கடை.txt

saving dinamalar_nellai_62911_உபி_அமைச்சர்களின்_தனி_செயலாளர்கள்_3_பேர்_ஊழ


saving dinamalar_nellai_62842_27122018_பருப்பு_மற்றும்_மாவு_எண்ணெய்_வகைகள்_மொத்த_மார்க்கெட்_விலை.txt

saving dinamalar_nellai_62841_மேகதாது_விவகாரத்தில்_தீர்வு_காண_கர்நாடக_–_தமிழக_முதல்வர்கள்_கூட்டம்_நிதின்_கட்கரி_தகவல்.txt

saving dinamalar_nellai_62840_27122018_இந்திய_ரூபாய்_நாணய_மாற்று_விகிதம்_துவக்க_நிலவரம்.txt

saving dinamalar_nellai_62839_பெட்ரோல்_–_5_காசுகள்_டீசல்_–_7_காசுகள்_இன்று_குறைப்பு.txt

saving dinamalar_nellai_62838_ரிசர்வ்_வங்கியின்_இருப்பு_தொகை_குறித்து_பரிந்துரை_வழங்க_உயர்நிலை_குழு_நியமனம்.txt

saving dinamalar_nellai_62837_ஆந்திர_பிரதேசத்துக்கான_உயர்நீதிமன்றம்_ஜனவரி_1ம்_தேதி_முதல்_செயல்படும்__ராம்நாத்_கோவிந்த்_உத்தரவு.txt

saving dinamalar_nellai_62836_ஒடிசாவிற்கு_நிதிச்_சுயாட்சி_கிடைக்க_வேண்டும்__முதல்வர்_நவின்_பட்னாயக்_வலியுறுத்தல்.txt

saving dinamalar_nellai_62835_நொய்டாவில்_பொது_இடங்களில்_வெள்ளிக்கிழமை_தொழுகை_நடத்த_தடை_மாயாவதி_கண்டனம்.txt

saving dinamalar_nellai_62834_மோடி-_சிவசேனா_மோதல்_வலுக்கிறது_பதவிக்காக_திருடர்கள்_புனிதராக்கப்படுகிறார்கள்”-_சிவசேனா_பதில

...


saving dinamalar_nellai_62430_அமெரிக்கா_–_நேபாளம்_வெளியுறவு_துறை_அமைச்சர்கள்_பேச்சுவார்த்தை.txt

saving dinamalar_nellai_62429_தளபதி_விஜய்_நடித்த_சர்கார்_படத்தின்_ஒருவிரல்_புரட்சியே_புரட்சிகர_வீடியோ_பாடல்.txt

saving dinamalar_nellai_62428_இந்தியாவில்_ஜான்ஸன்_அண்டு_ஜான்ஸன்_நிறுவனத்தின்_பேபி_பவுடர்_மாதிரிகள்_ஆய்வு.txt

saving dinamalar_nellai_62427_மேகாலயாவில்_நிலக்கரி_சுரங்கத்தினுள்_சிக்கிய_தொழிலாளர்களை_மீட்கும்_பணி_மீண்டும்_துவங்கியது.txt

saving dinamalar_nellai_62426_கல்வியால்_கிடைத்த_பயன்.txt

saving dinamalar_nellai_62425_அப்படி_சொல்லக்_கூடாது.txt

saving dinamalar_nellai_62424_சிறுகதை__அதிகாரக்_கொடி.txt

saving dinamalar_nellai_62423_பாறையின்_வயது_என்ன.txt

saving dinamalar_nellai_62422_கண்டு_பிடியுங்கள்.txt

saving dinamalar_nellai_62421_மொக்க_ஜோக்ஸ்.txt

saving dinamalar_nellai_62420_அணையாத_தீப_ஜோதி.txt

saving dinamalar_nellai_62419_பள்ளி_பாட_திட்டம்_பாதியாக_குறைகிறது.txt

saving dinamalar_nellai_62418_செப்பு_மொழி_இருபத்தியிரண்டு.txt

saving dinamalar_nellai_62417_அதிமேதாவி


saving dinamalar_nellai_62337_உர்ஜித்_படேல்_ராஜினாமா_செய்யவேண்டும்_என்று_மத்திய_அரசு_கேட்கவில்லை__அருண்_ஜெட்லி_தகவல்.txt

saving dinamalar_nellai_62336_திருப்பூர்_நாமக்கல்_மாவட்ட_பால்_உற்பத்தியாளர்_கூட்டுறவு_ஒன்றியங்களை_முதல்வர்_துவக்கி_வைத்தார்.txt

saving dinamalar_nellai_62335_ராகுல்_காந்தியை_பிரதமர்_வேட்பாளராக_ஸ்டாலின்_முன்மொழிந்தது_பெரும்_திருப்பம்_-_திருமாவளவன்.txt

saving dinamalar_nellai_62334_ஈரான்_மீதான_பொருளாதார_தடையில்_சபாபர்_துறைமுக_பணிகளுக்கு_அமெரிக்கா_விலக்கு__ஆப்கன்_நன்றி.txt

saving dinamalar_nellai_62333_ஜெயலலிதா_சிகிச்சை_பெற்ற_போது_உணவுக்காக_மட்டும்_ரூ117_கோடி_செலவு_அப்போலோ_மருத்துவமனை.txt

saving dinamalar_nellai_62332_விவசாயிகளின்_கடன்கள்_முழுக்க_ரத்து_செய்யப்படும்_வரை_பிரதமர்_மோடியை_தூங்கவிடமாட்டேன்__ராகுல்_காந்தி_அறிவிப்பு.txt

saving dinamalar_nellai_62331_ரஜினிகாந்த்_மீது_சினிமா_பைனான்சியர்_தொடர்ந்த_அவதூறு_வழக்கு_ரத்து.txt

saving dinamalar_nellai_62330_வங்கி_மொபைல்_எண்ணுக்கு_ஆதார்_கட்டாயமில்லை_சட்ட_திருத்தத்துக்கு_மத்திய_அமைச்சரவை_ஒப்புதல்.txt

saving dinamal


saving dinamalar_nellai_62255_ரபேல்_விவகாரத்தில்_வேண்டுமென்றே_மக்களை_தவறாக_வழிநடத்துகிறது_காங்கிரஸ்_நிர்மலா_சீதாராமன்_குற்றச்சாட்டு.txt

saving dinamalar_nellai_62254_எல்லை_தாண்டி_மீன்பிடித்ததாக_தமிழக_மீனவர்கள்_8_பேர்_இலங்கை_கடற்படையினரால்_கைது.txt

saving dinamalar_nellai_62253_எங்கள்_கடமையை_நிறைவேற்றுவோம்_ராஜஸ்தான்_மக்களிடம்_ராகுல்_உறுதி.txt

saving dinamalar_nellai_62252_மூத்த_பத்திரிகையாளர்_சுதாங்கனின்_சிவநேசச்_செல்வர்கள்_–_133.txt

saving dinamalar_nellai_62251_மூத்த_பத்திரிகையாளர்_சுதாங்கனின்_சிவநேசச்_செல்வர்கள்_–_133.txt

saving dinamalar_nellai_62250_அகத்தை_துடைத்து_அகமியத்தை_அறிந்துகொள்_மனிதனே.txt

saving dinamalar_nellai_62249_பெருமானாரின்_அருமை_மனைவி_கதீஜா_நாயகி_வாழ்க்கை_வரலாறு–_5.txt

saving dinamalar_nellai_62248_நபிமார்கள்_வரலாறு_228_ஹஜ்ரத்_ஈஸா_நபி_அலைஹிஸ்ஸலாம்.txt

saving dinamalar_nellai_62247_பெய்ட்டி_புயல்_ஆந்திராவின்_காக்கிநாடா_-_ஏனாம்_இடையே_கரையைக்_கடந்தது.txt

saving dinamalar_nellai_62246_கலைமாமணி_வாமனனின்_நிழலல்ல_நிஜம்_–_158.txt

saving dinamalar_nellai_62245_கல


saving dinamalar_nellai_62173_தற்போதைய_செய்தி_4.txt

saving dinamalar_nellai_62172_அஜித்_நடிக்கும்_விஸ்வாசம்_படத்தின்_வேட்டிகட்டு_பாடல்_வரிகள்_கொண்ட_வீடியோ_காட்சி.txt

saving dinamalar_nellai_62171_வெள்ளை_மாளிகையின்_இடைக்கால_தலைமை_அதிகாரியாக_மிக்_முல்வானே_அதிபர்_டிரம்ப்_அறிவிப்பு.txt

saving dinamalar_nellai_62170_பெண்களுக்கு_33_சதவீத_இட_ஒதுக்கீடு_வழங்கக்_கோரி_பஞ்சாப்_சட்டப்பேரவையில்_தீர்மானம்.txt

saving dinamalar_nellai_62169_பொதுப்பணித்_துறை_கட்டிடங்களில்_எல்ஈடி_பல்புகள்_பொருத்த_மத்திய_பொதுப்பணித்_துறை_உத்தரவு.txt

saving dinamalar_nellai_62168_ரபேல்_ஒப்பந்தம்_பற்றிய_மத்திய_தணிக்கை_அறிக்கை_எப்போது_தாக்கல்_செய்யப்பட்டது_மல்லிகார்ஜூன்_கார்கே_கேள்வி.txt

saving dinamalar_nellai_62167_தற்போதைய_செய்தி_4.txt

saving dinamalar_nellai_62166_ஜெயம்_ரவி_நடிக்கும்_அடங்கமறு_படத்தின்_ஐந்து_நிமிட_வீடியோ_காட்சி.txt

saving dinamalar_nellai_62165_ஸ்டெர்லைட்_ஆலை_திறந்தால்_ஆலையை_மூடக்கோரி_மீண்டும்_மக்கள்_போராட்டம்_ஜிகே_வாசன்_எச்சரிக்கை.txt

saving dinamalar_nellai_62164_வீடியோ_கான்பரன்ஸ்_மூலம்_தமிழக_


saving dinamalar_nellai_62093_செந்தில்_பாலாஜி_திமுகவிற்கு_சென்றதில்_வருத்தம்_இல்லை_டிடிவி_தினகரன்.txt

saving dinamalar_nellai_62092_செந்தில்_பாலாஜி_திமுகவில்_இணைவது_கடலில்_கரைத்த_பெருங்காயம்_போன்றது_ஜெயக்குமார்.txt

saving dinamalar_nellai_62091_கஅன்பழகன்_97-ம்_பிறந்த_நாள்_நிவாரணப்_பணிகளை_மேற்கொள்ள_தொண்டர்களுக்கு_ஸ்டாலின்_வேண்டுகோள்.txt

saving dinamalar_nellai_62090_திமுகவில்_இணைந்தது_ஏன்-_செந்தில்_பாலாஜி_விளக்கம்.txt

saving dinamalar_nellai_62089_மு_க_ஸ்டாலின்_முன்னிலையில்_திமுகவில்_மீண்டும்_இணைந்தார்_செந்தில்_பாலாஜி.txt

saving dinamalar_nellai_62088_ஹாரீஸ்_ஜெயராஜ்_இசையில்_தேவ்_படத்தின்_அணங்கே_பாடல்_வரிகள்_வீடியோ.txt

saving dinamalar_nellai_62087_ரபேல்_ஒப்பந்த_விவகாரம்_விசாரணை_கோரிய_மனுக்களை_தள்ளுபடி_செய்தது_உச்சநீதிமன்றம்.txt

saving dinamalar_nellai_62086_ராகுல்_மன்னிப்பு_கேட்கவேண்டும்_பாஜக_எம்பிக்கள்_தொடர்_அமளியால்_நாடாளுமன்றத்தின்_இரு_அவைகளும்_ஒத்திவைப்பு.txt

saving dinamalar_nellai_62085_14-12-2018_கோயம்பேடு_மார்க்கெட்_காய்கறி_மொத்தவிலை_பட்டியல்.txt

saving dinamalar_nella

# Debug one article retrieve

In [ ]:
# Get page html first.
#article_id=87698
article_id

In [154]:
    base_url='http://www.dinamalarnellai.com/web/news'
    url='{}/{}'.format(base_url,article_id)
    r=requests.get(url)
    html = r.text

In [156]:
    selector = 'body > div.container > div.dinamalar_middle_container_news > div'    
    pub_selector = 'body > div.container > div.dinamalar_middle_container_news > div > div > div > h4'
    text_selector = 'div.dinamalar_middle_container_news > div > div > p'
    #img_selector = 'div.dinamalar_middle_container_news > div > div > p > img'
    img_pattern='<img class="(.*?)" src="http://www.dinamalarnellai.com/site/news_folder/(.*?).jpg" style="(.*?)">'

    soup = BeautifulSoup(html, 'lxml')
    try:
        title_selector = 'body > div.container > div.dinamalar_middle_container_news > div > h1'
        title = soup.select(title_selector)[0].text
    except:
        title_pattern = '<h1(.*?)>\n(.*?)</h1>'
        title_class,title=re.findall(title_pattern, html)[0]
        title=title.replace('\t','')
        #print(title)
        assert title!=''
    try:
        pubdate = soup.select(pub_selector)[0].text.strip()
    except:
        pubdate = re.findall('<p class="pageMeta">(.*?),', html)[0]
        pubdate=pubdate.replace('\t','')
    text = '\n'.join([para.text.replace('\xa0',' ') for para in soup.select(text_selector)])
    if text=='':
        paras =soup.select('div > p') #, attrs={'class':'pageContent'})
        text = '\n'.join([para.text.replace('\xa0',' ') for para in paras])
   
    #imglinks = soup.select(img_selector)
    imglinks=re.findall(img_pattern, html)
    imglinks

IndexError: list index out of range

In [105]:
title

'பாதுகாப்பு உபகரணங்கள் இல்லாமல் கொரோனா சிகிச்சைக்கு அனுப்புவதாக பிரன்ஸ் செவிலியர் போராட்டம்'

In [157]:
loc=html.find('h1>')
html[loc-200:loc+500]

''

In [158]:
html

'<!DOCTYPE html>\n<html lang="en">\n<head>\n<meta charset="utf-8">\n<meta http-equiv="refresh" content="180">\n<title>Latest News | Tamil News | Headlines | Tamil Online Paper | Nellai News | India, World News |  Dinamalar</title>\n<meta http-equiv="X-UA-Compatible" content="IE=EmulateIE7" />\n<meta http-equiv="Content-Type" content="text/html; charset=UTF-8"/>\n<meta name="author" content="">\n<meta property="fb:admins" content="1832341473"/>\n<meta property="fb:app_id" content="1510522369268274" />\n\n<meta property="og:url" content="http://www.dinamalarnellai.com/web/index" />\n<meta property="og:type" content="website" />\n\n<meta property="og:image" content="http://www.dinamalarnellai.com/site/oglogo.jpg"/>\n<meta property="og:image:url" content="http://www.dinamalarnellai.com/site/oglogo.jpg"/>\n\n<meta name="description" content="Dinamalar, Dinamalar Nellai - Latest News | Tamil News | Headlines | Tamil Online Paper | Nellai News | India, World News  -  தினமலர்"/>\n\n\n<meta nam

In [75]:
title_pattern, title = re.findall('<h1(.*?)>\n(.*?)</h1>', html)[0]
title=title.replace('\t','')
title

'16 வயதில் மரணமடைந்த நடிகர்!'

In [77]:
pubdate

' 05 ஏப்ரல் 2020'

In [78]:
imglinks

[('fr-dib fr-draggable',
  '19297356301586075337767661773logan11',
  'width: 364px; height: 373.517px;')]

In [86]:
paras =soup.select('div > p') #, attrs={'class':'pageContent'})
text_selector

[<p class="pageMeta"> 05 ஏப்ரல் 2020, 01:59 PM 
 					<span class="readInLanguage">
 </span>
 </p>, <p>
 </p>, <p>தி பிளாஷ் என்கிற தொலைக்காட்சி தொடர் மூலமாகப் புகழ்பெற்ற கனடாவைச் சேர்ந்த 16 வயது நடிகர் லோகன் வில்லியம்ஸ் மரணமடைந்துள்ளார்.</p>, <p><img class="fr-dib fr-draggable" src="http://www.dinamalarnellai.com/site/news_folder/19297356301586075337767661773logan11.jpg" style="width: 364px; height: 373.517px;"/></p>, <p>லோகனின் தாய் மர்லிஸ் வில்லியம்ஸ் இத்தகவலைத் தெரிவித்துள்ளார். லோகனின் மரணத்துக்கான காரணம் தெரிவிக்கப்படவில்லை.</p>, <p>திறமை மற்றும் அழகான தோற்றத்தினால் பெரிய நடிகராக வருவதற்கான வாய்ப்புகள் இருந்தன என்று தனது வருத்தத்தை மர்லிஸ் பதிவு செய்துள்ளார். கரோனா காரணமாக அஞ்சலி செலுத்துவதற்கும் இறுதிச்சடங்குகளுக்கும் கூட சிரமமாக உள்ளது என்றும் அவர் கூறியுள்ளார்.</p>, <p>16 வயதில் ஒரு நடிகர் மரணமடைந்திருப்பது திரையுலகை அதிர்ச்சியடையவைத்துள்ளது. லோகனுடன் நடித்த பலரும் தங்களுடைய இரங்கல்களைத் தெரிவித்துள்ளார்கள்.</p>]

In [89]:
text

' 05 ஏப்ரல் 2020, 01:59 PM \n\t\t\t\t\t\n\n\n\n\nதி பிளாஷ் என்கிற தொலைக்காட்சி தொடர் மூலமாகப் புகழ்பெற்ற கனடாவைச் சேர்ந்த 16 வயது நடிகர் லோகன் வில்லியம்ஸ் மரணமடைந்துள்ளார்.\n\nலோகனின் தாய் மர்லிஸ் வில்லியம்ஸ் இத்தகவலைத் தெரிவித்துள்ளார். லோகனின் மரணத்துக்கான காரணம் தெரிவிக்கப்படவில்லை.\nதிறமை மற்றும் அழகான தோற்றத்தினால் பெரிய நடிகராக வருவதற்கான வாய்ப்புகள் இருந்தன என்று தனது வருத்தத்தை மர்லிஸ் பதிவு செய்துள்ளார். கரோனா காரணமாக அஞ்சலி செலுத்துவதற்கும் இறுதிச்சடங்குகளுக்கும் கூட சிரமமாக உள்ளது என்றும் அவர் கூறியுள்ளார்.\n16 வயதில் ஒரு நடிகர் மரணமடைந்திருப்பது திரையுலகை அதிர்ச்சியடையவைத்துள்ளது. லோகனுடன் நடித்த பலரும் தங்களுடைய இரங்கல்களைத் தெரிவித்துள்ளார்கள்.'

# Exploring automatic pattern extraction, did not work fully

In [55]:
expected = {'title':'காலக் கணிதம் கண்ணதாசன் - 9',
           'published_date':'பதிவு செய்த நாள் : 07 ஏப்ரல் 2020',
            'article':'மாடர்ன் தியேட்டர்ஸ்',
            'image_link':'http://www.dinamalarnellai.com/site/news_folder/'
           }

In [56]:
e = expected['title']
e = expected['published_date']
e

'பதிவு செய்த நாள் : 07 ஏப்ரல் 2020'

In [59]:
def find_locations(snippet, text):
    loc=0
    while True:
        loc=text.find(snippet,loc+1) # not using closing tag marker since there could be spaces!
        if loc==-1:
            break
        print(html[loc-100:loc+100])
        #i=loc
        #while True:
        #    i-=1
        #    if text[i]=='<':
        #        break
        #j=loc+len(snippet)
        #while True:
        #    j+=1
        #    if text[j]=='>':
        #        break
        #print(html[i:j+1])

In [60]:
for key in expected:
    e=expected[key]
    find_locations(e,html)


91017857464Tamil_News_Nellai.jpg"/>
<meta name="description" content="Dinamalar, Dinamalar Nellai - காலக் கணிதம் கண்ணதாசன் - 9  - &nbsp; மாடர்ன் தியேட்டர்ஸ் படங்களுக்கு பெரும்பாலும் வேதா தான் 1965 களி
news, latest news, dinamalar nellai, cinema news, sports news, " />

<meta itemprop="name" content="காலக் கணிதம் கண்ணதாசன் - 9  - தினமலர்">
<meta property="og:title" content="காலக் கணிதம் கண்ணதாசன் - 
itemprop="name" content="காலக் கணிதம் கண்ணதாசன் - 9  - தினமலர்">
<meta property="og:title" content="காலக் கணிதம் கண்ணதாசன் - 9  - தினமலர்">
<meta property="og:description" content="Dinamalar, Dinamala
் கண்ணதாசன் - 9  - தினமலர்">
<meta property="og:description" content="Dinamalar, Dinamalar Nellai - காலக் கணிதம் கண்ணதாசன் - 9  - &nbsp; மாடர்ன் தியேட்டர்ஸ் படங்களுக்கு பெரும்பாலும் வேதா தான் 1965 களி
ht:10px;"></div>
  <div style="margin-left:10px;">
  <h1 class="newsdetbd1" style="font-size:15px;">காலக் கணிதம் கண்ணதாசன் - 9</h1>
 
	
<div style="margin-right:10px; margin-bottom:5px;">
<div id

In [66]:
patterns = {'title':'<h1 class="newsdetbd1" style="font-size:15px;">காலக் கணிதம் கண்ணதாசன் - 9</h1>',
           'published_date':'பதிவு செய்த நாள் : 07 ஏப்ரல் 2020',
           'article':'<p>(.*?)</p>'}

In [67]:
for key in patterns:
    e=patterns[key]
    matches=re.findall(e,html)
    for m in matches:
        print(m)

<h1 class="newsdetbd1" style="font-size:15px;">காலக் கணிதம் கண்ணதாசன் - 9</h1>
பதிவு செய்த நாள் : 07 ஏப்ரல் 2020
<iframe src="https://www.youtube.com/embed/-oefNLWf8Gc?autoplay=1" width="100%" height="425" frameborder="0" style="position:absolute;left:0" allowfullscreen></iframe>
சினிமா 
கோயில்கள்
விளையாட்டு
வர்த்தகம்
இ-பேப்பர்
&nbsp; மாடர்ன் தியேட்டர்ஸ் படங்களுக்கு பெரும்பாலும் வேதா தான் 1965 களில் இசையமைத்து வந்தார். மாடர்ன் தியேட்டர்ஸ் மிகப் பிரபலமான இந்திப் பாடல்களின் உரிமைகளை வாங்கி அதை அப்படியே தமிழில் போடச் சொல்வார்கள்.
<img class="fr-dib fr-draggable" src="http://www.dinamalarnellai.com/site/news_folder/20321693021586237816191328358New Project (6).jpg" style="width: 300px;">
&nbsp;வேதாவிற்கு இதில் பெரும் மனக்குறை உண்டு. தன்னுடைய தனித்துவத்திற்கு மாடர்ன் தியேட்டர்ஸ் மதிப்பு கொடுப்பதில்லை என்று கண்ணதாசனிடம் சொல்லி வருத்தப்படுவார். அப்போது கண்ணதாசன் சொல்வார் ` கவலையை விடு. இசைதானே அவங்களுடையது. வார்த்தைகளில் அவர்கள் பாடலை நாம் மிஞ்சிக்காட்டுவோம். அந்த வார்த்தைகளை கேட்பவர்கள் உன்னை